[![Binder](https://mybinder.org/badge_logo.svg)](https://nbviewer.org/github/Sistemas-Multimedia/Sistemas-Multimedia.github.io/blob/master/milestones/07-DCT/JPEG.ipynb)

# [JPEG2000](https://pillow.readthedocs.io/en/stable/handbook/image-file-formats.html#jpeg-2000) RD curve

Unfinished

In [ ]:
import os

In [ ]:
kakadu="KDU805_Demo_Apps_for_Linux-x86-64_200602"

In [ ]:
%%bash -s "$kakadu"
if [ -f $1.zip ]; then
    echo "Kakadu already downloaded"
else
    wget http://kakadusoftware.com/wp-content/uploads/$1.zip
fi

In [ ]:
!unzip $kakadu

In [ ]:
!echo $LD_LIBRARY_PATH

In [ ]:
old = os.environ.get("LD_LIBRARY_PATH")
home = os.environ["HOME"]
if old:
    os.environ["LD_LIBRARY_PATH"] = old + ":" + kakadu
else:
    os.environ["LD_LIBRARY_PATH"] = home + "/repos/JPEG2000/" + kakadu

In [ ]:
!echo $LD_LIBRARY_PATH

In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
!ln -sf ~/repos/image_IO/logging_config.py .
!ln -sf ~/repos/image_IO/image_3.py .
import image_3 as image
!ln -sf ~/repos/information_theory/distortion.py .
!ln -sf ~/repos/information_theory/information.py .
import distortion
import pylab
import PIL
import subprocess


In [ ]:
HOME = os.environ["HOME"]
test_image = HOME + "/repos/MRVC/images/lena_color/"

In [ ]:
JPEG2K_RD_points = []
xx = image.read(test_image, 0)
for slope in range(45000, 43000, -100):
    !convert {test_image}000.png /tmp/1.tiff
    !./{kakadu}/kdu_compress -i /tmp/1.tiff -o /tmp/1.j2c -slope {slope}
    BPP = os.path.getsize("/tmp/1.j2c")*8/(xx.shape[0]*xx.shape[1])
    zz = np.asarray(PIL.Image.open("/tmp/1.j2c"))
    RMSE = distortion.RMSE(xx, zz)
    JPEG2K_RD_points.append((BPP, RMSE))
    print(BPP, RMSE)

In [ ]:
pylab.figure(dpi=150)
pylab.plot(*zip(*JPEG2K_RD_points), label="JPEG 2000")
pylab.title(f"JPEG 2000 {test_image}")
pylab.xlabel("Bits/Pixel")
pylab.ylabel("RMSE")
plt.legend(loc="best")
pylab.show()

In [ ]:
with open('JPEG2000.txt', 'w') as f:
    for item in JPEG2K_RD_points:
        f.write(f"{item[0]}\t{item[1]}\n")